In [2]:
import plotly.express as px
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.graph_objs as go

In [3]:
# df_pie.index.to_list()
# data_to_plot = data[data['iso_alpha'].isin(df_pie.index.to_list())]
# data_to_plot = data_to_plot[['share_pv', 'share_wind', 'iso_alpha']].groupby('iso_alpha').mean()
# data_to_plot[['share_wind','share_pv']].plot.barh(stacked=True)

In [4]:
country_codes = pd.read_csv('../countries_codes_and_coordinates.csv' , sep = ',', index_col = 0)

In [5]:
data = pd.read_csv('results/optimization_results_world_grid.csv')
data = data.set_index('Country')
data = data.drop(['Antarctica'], axis =0)
for col in data.columns[2:]: 
    data[f'{col}_normalized']=data[col]/data['mean_load']
    
for c in data.index.to_list(): 
    results = pd.read_pickle(f'results/{c}/optimization_results.pickle')
    iso_code = country_codes.loc[c,'Alpha-3 code' ].split(' ')[1]
    data.loc[c, 'capacity_factor_wind'] = results['E_wind']/data.loc[c, 'P_wind']/8760
    data.loc[c, 'capacity_factor_pv'] = results['E_pv']/data.loc[c, 'P_pv']/8760
    data.loc[c, 'capacity_factor_dispatch'] = results['E_dispatch']/data.loc[c, 'P_dispatch']/8760
    data.loc[c, 'share_dispatchable'] = results['E_dispatch']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    data.loc[c, 'share_wind']=results['E_wind']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    data.loc[c, 'share_pv']=results['E_pv']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    # data.loc[c, 'carbon_impact']=results['E_wind']*11+results['E_pv']*45/pop/1000
    data.loc[c, 'weight_wind']=results['wind_capacity']/results['mean_consumption']
    data.loc[c, 'weight_pv']=results['pv_capacity']/results['mean_consumption']
    data.loc[c, 'share_P_pv']=data.loc[c, 'weight_pv']/(data.loc[c, 'weight_pv']+data.loc[c, 'weight_wind'])
    data.loc[c, 'share_P_wind']=data.loc[c, 'weight_wind']/(data.loc[c, 'weight_pv']+data.loc[c, 'weight_wind'])

In [6]:
data_us = pd.read_csv('results/optimization_results_world_grid_with_states.csv')
data_us = data_us.set_index('State')

for col in data_us.columns[2:]: 
    data_us[f'{col}_normalized']=data_us[col]/data_us['mean_load']

for i in range(len(data_us.index.to_list())): 
    c = data_us.index[i]
    country_name = data_us.iloc[i,0]
    results = pd.read_pickle(f'./results/{country_name}/optimization_results_{c}.pickle')
    iso_code = country_codes.loc[country_name,'Alpha-3 code' ].split(' ')[1]
    # pop = pop_data.loc[iso_code]
    # pop=pop.iloc[0]
    data_us.loc[c, 'capacity_factor_wind'] = results['E_wind']/data_us.loc[c, 'P_wind']/8760
    data_us.loc[c, 'capacity_factor_pv'] = results['E_pv']/data_us.loc[c, 'P_pv']/8760
    data_us.loc[c, 'capacity_factor_dispatch'] = results['E_dispatch']/data_us.loc[c, 'P_dispatch']/8760
    data_us.loc[c, 'share_dispatchable'] = results['E_dispatch']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    data_us.loc[c, 'share_wind']=results['E_wind']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    data_us.loc[c, 'share_pv']=results['E_pv']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    # data_us.loc[c, 'carbon_impact']=results['E_wind']*11+results['E_pv']*45/pop/1000
    data_us.loc[c, 'weight_wind']=results['wind_capacity']/results['mean_consumption']
    data_us.loc[c, 'weight_pv']=results['pv_capacity']/results['mean_consumption']
    data_us.loc[c, 'share_P_pv']=data_us.loc[c, 'weight_pv']/(data_us.loc[c, 'weight_pv']+data_us.loc[c, 'weight_wind'])
    data_us.loc[c, 'share_P_wind']=data_us.loc[c, 'weight_wind']/(data_us.loc[c, 'weight_pv']+data_us.loc[c, 'weight_wind'])

In [7]:
data_us

,Country,iso_alpha,mean_load,E_dispatch,P_dispatch,E_destock,P_pv,P_wind,mean_load_normalized,E_dispatch_normalized,...,capacity_factor_wind,capacity_factor_pv,capacity_factor_dispatch,share_dispatchable,share_wind,share_pv,weight_wind,weight_pv,share_P_pv,share_P_wind
State,,,,,,,,,,,,,,,,,,,,,
CA,United States,USA,34194.854452,4.825488e+07,48992.507974,9.788962e+07,130332.936143,72748.670940,1.0,1411.173704,...,0.097208,0.208106,0.112437,13.874246,17.811451,68.314302,2.127474,3.811478,0.641776,0.358224
AL,United States,USA,34194.854452,4.892787e+07,56935.291358,8.011241e+07,129929.706300,58177.160455,1.0,1430.854848,...,0.138888,0.125917,0.082535,16.126394,27.728877,56.144729,2.668070,3.744256,0.583915,0.416085
AK,United States,USA,34194.854452,6.491029e+07,49895.279214,3.533449e+07,136590.876692,146077.004619,1.0,1898.247324,...,0.003279,0.001604,0.003155,18.401084,55.992761,25.606155,4.893038,3.126590,0.389867,0.610133
AZ,United States,USA,34194.854452,5.099935e+07,52283.136364,9.939957e+07,133803.541841,30217.171716,1.0,1491.433600,...,0.084244,0.105813,0.058774,15.538048,12.871791,71.590161,0.793054,3.995053,0.834370,0.165630
AR,United States,USA,34194.854452,4.474754e+07,53190.185329,8.373185e+07,130527.735635,46745.370763,1.0,1308.604543,...,0.127951,0.098265,0.052581,12.945627,27.684973,59.369400,1.373514,3.808789,0.734961,0.265039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CN051,China,CHN,23142.097374,3.423075e+07,27157.000000,6.930323e+07,102688.744856,497895.831719,1.0,1479.155158,...,0.007370,0.189629,0.143890,14.446066,13.565060,71.988874,21.514724,4.437314,0.170981,0.829019
CN045,China,CHN,15509.381050,1.894398e+07,17907.024399,4.142310e+07,69978.674370,15712.542932,1.0,1221.453183,...,0.252147,0.165015,0.120766,12.237228,22.418988,65.343784,1.013099,4.512022,0.816638,0.183362
CN022,China,CHN,7572.393950,9.934583e+06,8713.104974,1.701307e+07,29005.738669,8895.026052,1.0,1311.947459,...,0.319403,0.163116,0.130159,13.025758,32.631995,54.342247,1.174665,3.830458,0.765307,0.234693


In [8]:
# shapefile_path = './shapefiles/geoBoundariesCGAZ_ADM1.shp'
# gdf = gpd.read_file(shapefile_path)

# gdf = gdf.to_crs(epsg=4326)

# gdf['geometry'] = gdf['geometry'].simplify(tolerance=0.01)

# fig, ax = plt.subplots(figsize=(10, 10))
# gdf.plot(ax=ax, linewidth=0.5, edgecolor='k', color='lightblue')

# ax.set_title('Your Title Here')

# ax.set_axis_off()

# plt.show()

In [23]:
fig = go.Figure(go.Choropleth(
    locations=data['iso_alpha'],
    z=data['share_dispatchable'],
    text=data['iso_alpha'],
    hoverinfo='location+z+text',
    colorscale='Portland', zmin=0,  
    zmax=25 
))

fig.update_layout(
    title_text='Dispatchable energy (% of production)',
    geo=dict(
        projection=go.layout.geo.Projection(type='natural earth')
    )
)
# Save the figure as an HTML file without displaying it
fig.write_html('figures/world_maps/choropleth_map_dispatch_corrected_only_country.html')

In [24]:
fig = go.Figure(go.Choropleth(
    locations=data['iso_alpha'],
    z=data['share_pv'],
    text=data['iso_alpha'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100 
))

fig.update_layout(
    title_text='PV energy (% of production)',
    geo=dict(
        projection=go.layout.geo.Projection(type='natural earth')
    )
)
# Save the figure as an HTML file without displaying it
fig.write_html('figures/world_maps/choropleth_map_pv_share_corrected_only_country.html')

In [25]:
fig = go.Figure(go.Choropleth(
    locations=data['iso_alpha'],
    z=data['share_wind'],
    text=data['iso_alpha'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100 
))

fig.update_layout(
    title_text='Wind energy (% of production)',
    geo=dict(
        projection=go.layout.geo.Projection(type='natural earth')
    )
)
# Save the figure as an HTML file without displaying it
fig.write_html('figures/world_maps/choropleth_map_wind_share_corrected_only_country.html')

### Chargement des shapefiles

In [9]:
world = gpd.read_file('shapefiles/ne_110m_admin_0_countries.shp')

usa_gdf = gpd.read_file('shapefiles/USA/States_shapefile.shp')
usa_gdf = usa_gdf.merge(data_us[data_us['iso_alpha']=='USA'], left_on='State_Code', right_on='State')  

canada_gdf = gpd.read_file('shapefiles/Canada/Canada_Provincial_boundaries_generalized.shp')
canada_gdf = canada_gdf.to_crs(world.crs)
canada_gdf = canada_gdf.merge(data_us[data_us['iso_alpha']=='CAN'], left_on='postal', right_on='State')  

china_gdf = gpd.read_file('shapefiles/China/chn_admbnda_adm1_ocha_2020.shp')
china_gdf = china_gdf.merge(data_us[data_us['iso_alpha']=='CHN'], left_on='ADM1_PCODE', right_on='State')
brazil_gdf = gpd.read_file('shapefiles/Brazil/geoBoundaries-BRA-ADM1.shp')
for i in brazil_gdf.index :
    brazil_gdf.loc[i,'shapeISO'] = brazil_gdf.loc[i,'shapeISO'].split('-')[1]
brazil_gdf = brazil_gdf.merge(data_us[data_us['iso_alpha']=='BRA'], left_on='shapeISO', right_on='State')

# india_gdf = gpd.read_file('shapefiles/China/chn_admbnda_adm1_ocha_2020.shp')
# india_gdf = india_gdf.merge(data_us, left_on='State_Code', right_on='State')
australia_gdf = gpd.read_file('shapefiles/Australia/georef-australia-state-millesime.shp')
australia_gdf = australia_gdf.merge(data_us[data_us['iso_alpha']=='AUS'], left_on='ste_iso3166', right_on='State')

russia_gdf = gpd.read_file('shapefiles/Russia/geoBoundaries-RUS-ADM1.shp')
russia_gdf["geometry"]=(russia_gdf.to_crs(russia_gdf.estimate_utm_crs()).simplify(1000).to_crs(russia_gdf.crs))
for i in russia_gdf.index :
    russia_gdf.loc[i,'shapeISO'] = russia_gdf.loc[i,'shapeISO'].split('-')[1]
russia_gdf = russia_gdf.to_crs(world.crs)
russia_gdf = russia_gdf.merge(data_us[data_us['iso_alpha']=='RUS'], left_on='shapeISO', right_on='State') 

### Carte Capacité PV

In [13]:
fig = go.Figure(go.Choropleth(
    locations=data['iso_alpha'],
    z=data['share_P_pv']*100,
    text=data['iso_alpha'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100 
))

# Adding Canada provinces
fig.add_trace(go.Choropleth(
    geojson=canada_gdf.geometry.__geo_interface__,
    locations=canada_gdf.index,
    z=canada_gdf['share_P_pv']*100,
    text=canada_gdf['Name_EN'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding Brazil states
fig.add_trace(go.Choropleth(
    geojson=brazil_gdf.geometry.__geo_interface__,
    locations=brazil_gdf.index,
    z=brazil_gdf['share_P_pv']*100,
    text=brazil_gdf['shapeISO'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding Russia regions
fig.add_trace(go.Choropleth(
    geojson=russia_gdf.geometry.__geo_interface__,
    locations=russia_gdf.index,
    z=russia_gdf['share_P_pv']*100,
    text=russia_gdf['shapeISO'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding Australia states
fig.add_trace(go.Choropleth(
    geojson=australia_gdf.geometry.__geo_interface__,
    locations=australia_gdf.index,
    z=australia_gdf['share_P_pv']*100,
    text=australia_gdf['ste_code'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding China provinces
fig.add_trace(go.Choropleth(
    geojson=china_gdf.geometry.__geo_interface__,
    locations=china_gdf.index,
    z=china_gdf['share_P_pv']*100,
    text=china_gdf['ADM1_PCODE'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding USA states
fig.add_trace(go.Choropleth(
    geojson=usa_gdf.geometry.__geo_interface__,
    locations=usa_gdf.index,
    z=usa_gdf['share_P_pv']*100,
    text=usa_gdf['State_Name'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Update layout and map projection
fig.update_layout(
    title_text='PV capacity (% renewable capacity)',
    geo=dict(
        projection=go.layout.geo.Projection(type='natural earth')
    )
)

# Save the figure as an HTML file
fig.write_html('figures/world_maps/choropleth_map_share_pv_capacity.html')


### Carte Wind capacité

In [12]:
fig = go.Figure(go.Choropleth(
    locations=data['iso_alpha'],
    z=data['share_P_wind']*100,
    text=data['iso_alpha'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100 
))

# Adding Canada provinces
fig.add_trace(go.Choropleth(
    geojson=canada_gdf.geometry.__geo_interface__,
    locations=canada_gdf.index,
    z=canada_gdf['share_P_wind']*100,
    text=canada_gdf['Name_EN'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding Brazil states
fig.add_trace(go.Choropleth(
    geojson=brazil_gdf.geometry.__geo_interface__,
    locations=brazil_gdf.index,
    z=brazil_gdf['share_P_wind']*100,
    text=brazil_gdf['shapeISO'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding Russia regions
fig.add_trace(go.Choropleth(
    geojson=russia_gdf.geometry.__geo_interface__,
    locations=russia_gdf.index,
    z=russia_gdf['share_P_wind']*100,
    text=russia_gdf['shapeISO'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding Australia states
fig.add_trace(go.Choropleth(
    geojson=australia_gdf.geometry.__geo_interface__,
    locations=australia_gdf.index,
    z=australia_gdf['share_P_wind']*100,
    text=australia_gdf['ste_code'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding China provinces
fig.add_trace(go.Choropleth(
    geojson=china_gdf.geometry.__geo_interface__,
    locations=china_gdf.index,
    z=china_gdf['share_P_wind']*100,
    text=china_gdf['ADM1_PCODE'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding USA states
fig.add_trace(go.Choropleth(
    geojson=usa_gdf.geometry.__geo_interface__,
    locations=usa_gdf.index,
    z=usa_gdf['share_P_wind']*100,
    text=usa_gdf['State_Name'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Update layout and map projection
fig.update_layout(
    title_text='Wind capacity (% renewable capacity)',
    geo=dict(
        projection=go.layout.geo.Projection(type='natural earth')
    )
)

# Save the figure as an HTML file
fig.write_html('figures/world_maps/choropleth_map_share_wind_capacity.html')

### Carte énergie pilotable

In [28]:
fig = go.Figure(go.Choropleth(
    locations=data['iso_alpha'],
    z=data['share_dispatchable'],
    text=data['iso_alpha'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=20 
))

# Adding Canada provinces
fig.add_trace(go.Choropleth(
    geojson=canada_gdf.geometry.__geo_interface__,
    locations=canada_gdf.index,
    z=canada_gdf['share_dispatchable'],
    text=canada_gdf['Name_EN'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=20
))

# Adding Brazil states
fig.add_trace(go.Choropleth(
    geojson=brazil_gdf.geometry.__geo_interface__,
    locations=brazil_gdf.index,
    z=brazil_gdf['share_dispatchable'],
    text=brazil_gdf['shapeISO'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=20
))

# Adding Russia regions
fig.add_trace(go.Choropleth(
    geojson=russia_gdf.geometry.__geo_interface__,
    locations=russia_gdf.index,
    z=russia_gdf['share_dispatchable'],
    text=russia_gdf['shapeISO'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=20
))

# Adding Australia states
fig.add_trace(go.Choropleth(
    geojson=australia_gdf.geometry.__geo_interface__,
    locations=australia_gdf.index,
    z=australia_gdf['share_dispatchable'],
    text=australia_gdf['ste_code'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=20
))

# Adding China provinces
fig.add_trace(go.Choropleth(
    geojson=china_gdf.geometry.__geo_interface__,
    locations=china_gdf.index,
    z=china_gdf['share_dispatchable'],
    text=china_gdf['ADM1_PCODE'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=20
))

# Adding USA states
fig.add_trace(go.Choropleth(
    geojson=usa_gdf.geometry.__geo_interface__,
    locations=usa_gdf.index,
    z=usa_gdf['share_dispatchable'],
    text=usa_gdf['State_Name'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=20
))

# Update layout and map projection
fig.update_layout(
    title_text='Dispatchable Energy (% of production)',
    geo=dict(
        projection=go.layout.geo.Projection(type='natural earth')
    )
)

# Save the figure as an HTML file
fig.write_html('figures/world_maps/choropleth_map_dispatch_with_states_corrected.html')


### Carte énerie PV

In [29]:
fig = go.Figure(go.Choropleth(
    locations=data['iso_alpha'],
    z=data['share_pv'],
    text=data['iso_alpha'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding Canada provinces
fig.add_trace(go.Choropleth(
    geojson=canada_gdf.geometry.__geo_interface__,
    locations=canada_gdf.index,
    z=canada_gdf['share_pv'],
    text=canada_gdf['Name_EN'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding Brazil states
fig.add_trace(go.Choropleth(
    geojson=brazil_gdf.geometry.__geo_interface__,
    locations=brazil_gdf.index,
    z=brazil_gdf['share_pv'],
    text=brazil_gdf['shapeISO'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding Russia regions
fig.add_trace(go.Choropleth(
    geojson=russia_gdf.geometry.__geo_interface__,
    locations=russia_gdf.index,
    z=russia_gdf['share_pv'],
    text=russia_gdf['shapeISO'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding Australia states
fig.add_trace(go.Choropleth(
    geojson=australia_gdf.geometry.__geo_interface__,
    locations=australia_gdf.index,
    z=australia_gdf['share_pv'],
    text=australia_gdf['ste_code'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding China provinces
fig.add_trace(go.Choropleth(
    geojson=china_gdf.geometry.__geo_interface__,
    locations=china_gdf.index,
    z=china_gdf['share_pv'],
    text=china_gdf['ADM1_PCODE'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding USA states
fig.add_trace(go.Choropleth(
    geojson=usa_gdf.geometry.__geo_interface__,
    locations=usa_gdf.index,
    z=usa_gdf['share_pv'],
    text=usa_gdf['State_Name'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Update layout and map projection
fig.update_layout(
    title_text='PV Energy (% of production)',
    geo=dict(
        projection=go.layout.geo.Projection(type='natural earth')
    )
)


# Save the figure as an HTML file
fig.write_html('figures/world_maps/choropleth_map_pv_with_states_corrected.html')





### Carte énergie éolienne

In [30]:
fig = go.Figure(go.Choropleth(
    locations=data['iso_alpha'],
    z=data['share_wind'],
    text=data['iso_alpha'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding Canada provinces
fig.add_trace(go.Choropleth(
    geojson=canada_gdf.geometry.__geo_interface__,
    locations=canada_gdf.index,
    z=canada_gdf['share_wind'],
    text=canada_gdf['Name_EN'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding Brazil states
fig.add_trace(go.Choropleth(
    geojson=brazil_gdf.geometry.__geo_interface__,
    locations=brazil_gdf.index,
    z=brazil_gdf['share_wind'],
    text=brazil_gdf['shapeISO'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding Russia regions
fig.add_trace(go.Choropleth(
    geojson=russia_gdf.geometry.__geo_interface__,
    locations=russia_gdf.index,
    z=russia_gdf['share_wind'],
    text=russia_gdf['shapeISO'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding Australia states
fig.add_trace(go.Choropleth(
    geojson=australia_gdf.geometry.__geo_interface__,
    locations=australia_gdf.index,
    z=australia_gdf['share_wind'],
    text=australia_gdf['ste_code'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding China provinces
fig.add_trace(go.Choropleth(
    geojson=china_gdf.geometry.__geo_interface__,
    locations=china_gdf.index,
    z=china_gdf['share_wind'],
    text=china_gdf['ADM1_PCODE'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Adding USA states
fig.add_trace(go.Choropleth(
    geojson=usa_gdf.geometry.__geo_interface__,
    locations=usa_gdf.index,
    z=usa_gdf['share_wind'],
    text=usa_gdf['State_Name'],
    hoverinfo='location+z+text',
    colorscale='Portland', 
    zmin=0,  
    zmax=100
))

# Update layout and map projection
fig.update_layout(
    title_text='Wind Energy (% of production)',
    geo=dict(
        projection=go.layout.geo.Projection(type='natural earth')
    )
)

# Save the figure as an HTML file
fig.write_html('figures/world_maps/choropleth_map_wind_with_states_corrected.html')



